In [2]:
pip install mp-api pymatgen pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 88.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.8/306.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.2 MB/s eta 0:00:00
  Created wheel for bibtexparser: filename=bibtexparser-1.4.3-py3-none-any.whl 

In [3]:
from mp_api.client import MPRester
import pandas as pd
from tqdm import tqdm

In [ ]:
API_KEY = "YOUR_API_KEY"

mpr = MPRester(API_KEY)

In [5]:
fields = [
    "material_id",
    "formula_pretty",
    "composition",
    "formation_energy_per_atom",
    "energy_above_hull"
]


In [6]:
docs = mpr.summary.search(
    fields=fields,
    all_fields=False
)

/tmp/ipykernel_56/3151981947.py:1: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  docs = mpr.summary.search(
/usr/local/lib/python3.12/dist-packages/mp_api/client/core/client.py:515: UserWarning: Ignoring `fields` argument: All fields are always included when no query is provided.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/mp_api/client/core/client.py:1326: UserWarning: Omitting a query also includes deprecated documents in the results. Make sure to post-filter them out.
  warnings.warn(


Retrieving SummaryDoc documents:   0%|          | 0/164969 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2913: UserWarning: Not all sites have property velocities. Missing values are set to None.
  return cls.from_sites(sites, charge=charge, properties=dct.get("properties"))
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2913: UserWarning: Not all sites have property selective_dynamics. Missing values are set to None.
  return cls.from_sites(sites, charge=charge, properties=dct.get("properties"))
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2913: UserWarning: Not all sites have property coordination_no. Missing values are set to None.
  return cls.from_sites(sites, charge=charge, properties=dct.get("properties"))
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2913: UserWarning: Not all sites have property forces. Missing values are set to None.
  return cls.from_sites(sites, charge=charge, properties=dct.get("properties"))
/usr/local/lib/python3.12/dist-packages

In [7]:
records = []

for d in tqdm(docs):
    records.append({
        "material_id": d.material_id,
        "formula": d.formula_pretty,
        "composition": d.composition.reduced_formula,
        "formation_energy_per_atom": d.formation_energy_per_atom,
        "energy_above_hull": d.energy_above_hull
    })

df = pd.DataFrame(records)
print(df.shape)



 11%|█         | 22392/210579 [00:00<00:06, 27642.51it/s]/usr/local/lib/python3.12/dist-packages/pymatgen/core/composition.py:1366: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms: list[str] = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])

 27%|██▋       | 57679/210579 [00:02<00:05, 28439.75it/s]/usr/local/lib/python3.12/dist-packages/pymatgen/core/composition.py:1366: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms: list[str] = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])

 57%|█████▋    | 120040/210579 [00:04<00:03, 29731.76it/s]/usr/local/lib/python3.12/dist-packages/pymatgen/core/composition.py:1366: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to av

(210579, 5)


In [8]:
df = df.dropna(
    subset=["formation_energy_per_atom", "energy_above_hull"]
)


In [9]:
print(df.shape)

(200487, 5)


In [10]:
df = df.sort_values("energy_above_hull")
df = df.drop_duplicates(subset="composition", keep="first")


In [11]:
print(df.shape)

(150201, 5)


In [12]:
STABILITY_THRESHOLD = 0.05  # eV/atom

df["is_stable"] = (df["energy_above_hull"] <= STABILITY_THRESHOLD).astype(int)

In [13]:
from pymatgen.core import Composition

def extract_elements_and_fractions(formula):
    comp = Composition(formula)
    el_amt = comp.get_el_amt_dict()
    total = sum(el_amt.values())
    return (
        list(el_amt.keys()),
        [v / total for v in el_amt.values()]
    )

df["elements"], df["fractions"] = zip(
    *df["formula"].apply(extract_elements_and_fractions)
)

In [14]:
df.to_csv("mp_model2_composition_dataset.csv", index=False)